# ANALISIS DE RESULTADOS

## globales

In [ ]:
file_path = '../outputs/table_outputs/jetson_orin_agx/jetson_orin_agx_dynamic_batch.md'
name = 'orin_agx'
name_titulo = 'Jetson Orin AGX'

## funciones

In [ ]:
def generate_latex_table(data):
    latex_header = r"""
\begin{table}[]
    \centering
    \footnotesize
    \caption{Jetson Orin AGX}
    \begin{tabular}{@{}clccccccccccc@{}}
    \toprule
    & & \multicolumn{2}{c}{Latency {[}ms{]}} &  & Thr. {[}inf/s{]} &  \multicolumn{3}{c}{Model properties} &  & \multicolumn{2}{c}{Accuracy [\%]} \\ \cmidrule(lr){3-4}  \cmidrule(lr){7-9} \cmidrule(l){11-12} 
    & & ave.              & max.             &  &                       &  Size {[}MB{]}   & \# Layers   & \# Weights   &  & Top 1       & Top 5      \\ \midrule"""
    latex_footer = r"""
    \bottomrule
    \end{tabular}
\end{table}
    """

    latex_content = ""
    is_first = True
    for model_name, variants in data.items():
        if variants:
            model_section = f" "
            for i, variant in enumerate(variants):
                variant_name = variant['variant'].replace('_', ' ')  # Replace underscores with spaces
                if i == 0:
                    if is_first:
                        model_section += f" \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                        is_first = False
                    else:
                        model_section += f"   \\midrule \n"
                        model_section += f"    \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                   
                else:
                    model_section += f"    & {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                
                model_section += "\n"
            latex_content += model_section

    full_latex = latex_header + latex_content + latex_footer
    return full_latex

In [ ]:
def parse_markdown_data(file_content):
    models_data = file_content.split('# ')[1:]  # Divide por modelo y batch size, ignorando el primer split que estaría vacío
    data_summary = {}
    inf_s_data = {}

    # Procesar y almacenar inf/s para batch size 256 por separado
    best_inf_s = {}
    best_bs = {}
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, _ = model_name_batch_size.split(' bs ')
        for line in lines:
            if '|' in line and 'Model' not in line and not all(char == '-' for char in line.replace('|', '').strip()):
                cols = [col.strip() for col in line.split('|')]
                if len(cols) > 1:
                    variant = cols[1]
                    if model_name not in best_inf_s:
                            best_inf_s[model_name] = {}
                            best_bs[model_name] = {}
                    best_inf_s[model_name][variant] = 0.0
                    best_bs[model_name][variant] = 0

    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        #if batch_size == 256:
        for line in lines:
            if '|' in line and 'Model' not in line and not all(char == '-' for char in line.replace('|', '').strip()):
                cols = [col.strip() for col in line.split('|')]
                if len(cols) > 1:
                    variant = cols[1]
                    inf_s_str = (cols[2].split(' ')[0].replace('.','')).replace(',','.')
                    inf_s = float(inf_s_str)
                    if inf_s > best_inf_s[model_name][variant]:
                        best_inf_s[model_name][variant]  = inf_s
                        best_bs[model_name][variant] = batch_size
                        if model_name not in inf_s_data:
                            inf_s_data[model_name] = {}
                        if batch_size == 256:
                            inf_s_data[model_name][variant] = '$'+str(inf_s)+'$'
                        else: 
                            inf_s_data[model_name][variant] = '$'+str(inf_s)+'_{bs= '+str(batch_size)+'}$'

    # Procesar datos principales para batch size 1 y añadir inf/s de batch size 256 cuando corresponda
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        if batch_size == 1:
            table_data = [line for line in lines if '|' in line and 'Model' not in line]

            data_summary[model_name] = []

            for data in table_data:
                if not all(char == '-' for char in data.replace('|', '').strip()):
                    cols = [col.strip() for col in data.split('|')]
                    if len(cols) > 1:
                        model_variant = cols[1]
                        inf_s = inf_s_data.get(model_name, {}).get(model_variant, 'N/A')  # Get inf/s from bs 256 data if available
                        lat_mean = cols[3].split(' / ')[0]
                        lat_worst = (cols[3].split(' / ')[1]).split(' ')[0]
                        lat_mean = lat_mean.strip()
                        lat_worst = lat_worst.strip()
                        size_mb = cols[4]
                        prec_1 = cols[5]
                        prec_5 = cols[6]
                        layers = cols[7]
                        params = cols[8]

                        model_data = {
                            'variant': model_variant,
                            'inf_s': inf_s,  # Added from bs 256 data
                            'lat_mean': lat_mean,
                            'lat_worst': lat_worst,
                            'size_mb': size_mb.strip(),
                            'prec_1': prec_1.strip(),
                            'prec_5': prec_5.strip(),
                            'layers': layers.strip(),
                            'params': params.strip()
                        }
                        data_summary[model_name].append(model_data)

    return data_summary

In [ ]:
def parse_markdown_data_for_throughput(file_content):
    models_data = file_content.split('# ')[1:]  # Divide por modelo y batch size, ignorando el primer split que estaría vacío
    data_summary = {}

    # Procesar datos principales para batch size 1 y añadir inf/s de batch size 256 cuando corresponda
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        table_data = [line for line in lines if '|' in line and 'Model' not in line]

        for data in table_data:
            if not all(char == '-' for char in data.replace('|', '').strip()):
                cols = [col.strip() for col in data.split('|')]
                if len(cols) > 1:
                    model_variant = cols[1]
                    inf_s_str = (cols[2].split(' ')[0].replace('.','')).replace(',','.')
                    inf_s = float(inf_s_str)

                    if model_name not in data_summary:
                        data_summary[model_name] = {}
                    if model_variant not in data_summary[model_name]:
                        data_summary[model_name][model_variant] = {}
                    data_summary[model_name][model_variant][batch_size] = inf_s

    return data_summary

In [ ]:
def plot_network_latency(data):
    import matplotlib.pyplot as plt
    networks = list(data.keys())  # Networks as the categories
    variants = [entry['variant'] for entry in data['mobilenet']]  # Assuming all have the same variants

    # Construct a dictionary for lat_mean values, indexed by network and variant
    lat_means = {network: {entry['variant']: float(entry['lat_mean']) for entry in entries} for network, entries in data.items()}

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 8))
    width = 0.15  # Bar width

    # Positioning each bar for each variant within the group
    for i, variant in enumerate(variants):
        means = [lat_means[network][variant] for network in networks]
        ax.bar([x + i * width for x in range(len(networks))], means, width, label=variant)

    ax.set_xlabel('Network', fontweight='bold')
    ax.set_ylabel('Latency [ms]', fontweight='bold')
    ax.set_title(f'Mean Latency {name_titulo}')
    ax.set_xticks([x + width * (len(variants) - 1) / 2 for x in range(len(networks))])
    #ax.set_xticklabels(networks, rotation=20)
    ax.set_xticklabels(networks, fontweight='bold')
    ax.legend(title='Optimization')

    ax.grid(which='major', linestyle='-', linewidth='0.5', color='grey')  # Ajusta la grilla principal
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='lightgrey')  # Ajusta la grilla secundaria
    ax.minorticks_on()  # Activa las marcas menores

    plt.grid(True)
    plt.savefig(f'latency_{name}.pdf', format='pdf')
    plt.show()

In [ ]:
def plot_network_throughput(data):
    import matplotlib.pyplot as plt
    for network_name, variants in data.items():
        plt.figure(figsize=(10, 5))
        for variant_name, batch_sizes in variants.items():
            batch_sizes_sorted = sorted(batch_sizes.items())  # Ordenar los datos por batch size
            batch_sizes, performances = zip(*batch_sizes_sorted)
            plt.plot(batch_sizes, performances, label=variant_name, marker='o')
        plt.title(f'Inference Throughput {network_name}', fontweight='bold')
        plt.xlabel('Batch Size', fontweight='bold')
        plt.ylabel('Inference Throughput [inf/s]', fontweight='bold')
        plt.legend()
        plt.grid(True)
        plt.grid(which='major', linestyle='-', linewidth='0.5', color='grey')  # Ajusta la grilla principal
        plt.grid(which='minor', linestyle=':', linewidth='0.5', color='lightgrey')  # Ajusta la grilla secundaria
        plt.minorticks_on()
        plt.savefig(f'througput_{name}_{network_name}.pdf', format='pdf')
        plt.show()


### Path a archivo .md que contiene las tablas con resultados experimentales

In [ ]:
# Abre y lee el archivo
with open(file_path, 'r', encoding='utf-8') as file:
    md_content = file.read()
# Parse the markdown content
data = parse_markdown_data(md_content)

data_throughput = parse_markdown_data_for_throughput(md_content)

#print(data)
#print(data_throughput)

#for model_data in parsed_data.items():
#    for variants in model_data[1]:
#        print(model_data[0], variants)

### Generar tabla LaTex en para el escrito

In [ ]:
latex_table = generate_latex_table(data)
#print(latex_table)

# GRAFICOS

## Latencia

In [ ]:
plot_network_latency(data)

## Throughput

In [ ]:

plot_network_throughput(data_throughput)